# Tensorboard visualizations
Tensorboar is great tool for visualizing neural networks, its' topology and for logging values during training and evaliation. It's by default prepared for networks build with tensorflow  framework.

More details about tensorboard can be found https://www.tensorflow.org/guide/summaries_and_tensorboard

In [ ]:
%load_ext tensorboard.notebook
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

In [ ]:
!rm -rf tensorboard_logs
!rm -rf /tmp/.tensorboard-info

---

## Load and preprocess data

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [ ]:
train_images = train_images.reshape((-1, 28, 28, 1))
test_images = test_images.reshape((-1, 28, 28, 1))
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255

In [ ]:
test_labels = keras.utils.to_categorical(test_labels, 10)
train_labels = keras.utils.to_categorical(train_labels, 10)

In [ ]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

In [ ]:
# Create random batch generator
def data_generator():
    while True:
        idxs = np.random.randint(0, train_images.shape[0], 32)
        images, labels = train_images[idxs], train_labels[idxs]
        yield (images, labels)

---

## Prepare model

In [ ]:
log_dir_simple_cnn = './tensorboard_logs/simple_cnn/'
if not os.path.exists(log_dir_simple_cnn):
    os.makedirs(log_dir_simple_cnn)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard

tf.keras.backend.clear_session()

model = tf.keras.Sequential([
    Conv2D(filters=32, kernel_size=2, strides=1, padding='same', activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=2, strides=2),
    Dropout(rate=0.3),
    Conv2D(filters=32, kernel_size=2, strides=1, padding='same', activation='relu'),
    MaxPooling2D(pool_size=2, strides=2),
    Dropout(rate=0.3),
    Flatten(),
    Dense(128, activation='relu', name='embedding'),
    Dense(10, activation='softmax')
])

tensorboard = TensorBoard(log_dir=log_dir_simple_cnn, histogram_freq=1, write_graph=True)
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['mse', 'accuracy'])


In [ ]:
model.fit(data_generator(),
          batch_size=32,
          steps_per_epoch=100,
          epochs=5,
          verbose=1,
          validation_data=(test_images, test_labels),  callbacks=[tensorboard])

---

## Tensorboard with histograms and scalars

In [ ]:
%tensorboard --logdir tensorboard_logs/simple_cnn/

## Tensorboard with embedding projection

In [ ]:
from image_processing_workshop.tb_projection import get_sprite_img, setup_embedding_projection
import matplotlib.pylab as plt

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(get_sprite_img(test_images[:1000]))

In [ ]:
setup_embedding_projection(model, test_images[:1000], test_labels[:1000], log_dir_simple_cnn)

In [ ]:
%tensorboard --logdir tensorboard_logs/simple_cnn

## Tensorboard Resnet 50 visualization

In [ ]:
resnet50 = tf.keras.applications.resnet50.ResNet50()

In [ ]:
resnet50.summary()

In [ ]:
resnet50_tb = TensorBoard(log_dir="./tensorboard_logs/resnet/", histogram_freq=0, write_graph=True, write_images=True)

In [ ]:
resnet50_tb.set_model(resnet50)

In [ ]:
%tensorboard --logdir tensorboard_logs/resnet